In [33]:
import pandas as pd
import requests
DATA_DIR_URL = "https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/monthly/kl/historical"
STATION_INFO = DATA_DIR_URL + "/KL_Monatswerte_Beschreibung_Stationen.txt"

# df = pd.read_csv(STATION_INFO, encoding='windows-1252', sep=None, skiprows=[0,1,2])
# Can't parse the headers along with the rest, add manually
df = pd.read_fwf(
    STATION_INFO,
    encoding="windows-1252",
    colspecs="infer",
    infer_nrows=10,
    skiprows=[0, 1],
    header=None,
    names=[
        "Stations_id",
        "von_datum",
        "bis_datum",
        "Stationshoehe",
        "geoBreite",
        "geoLaenge",
        "Stationsname",
        "Bundesland",
    ],
)
df


,Stations_id,von_datum,bis_datum,Stationshoehe,geoBreite,geoLaenge,Stationsname,Bundesland
0,1,19310101,19860630,478,47.8413,8.8493,Aach,Baden-Württemberg
1,3,18510101,20110331,202,50.7827,6.0941,Aachen,Nordrhein-Westfalen
2,44,19710301,20221130,44,52.9336,8.2370,Großenkneten,Niedersachsen
3,52,19730101,20011231,46,53.6623,10.1990,Ahrensburg-Wulfsdorf,Schleswig-Holstein
4,61,19750701,19780831,339,48.8443,12.6171,Aiterhofen,Bayern
...,...,...,...,...,...,...,...,...
1119,19172,20200901,20221130,48,54.0246,9.3880,Wacken,Schleswig-Holstein
1120,19318,19461101,19781231,292,50.7180,10.4310,Schmalkalden,Thüringen
1121,19364,19371201,19441231,720,50.6167,10.8167,Schmiedefeld/Rennsteig,Thüringen
1122,19378,19580101,19771231,505,50.8333,10.5833,Finsterbergen,Thüringen


In [29]:
some_stn = df.iloc[50]
some_stn

Stations_id                      268
von_datum                   19530901
bis_datum                   19961231
Stationshoehe                    622
geoBreite                    48.5293
geoLaenge                     8.2727
Stationsname     Baiersbronn-Obertal
Bundesland         Baden-Württemberg
Name: 50, dtype: object

In [32]:
df[df['Stations_id'] == 19172]

,Stations_id,von_datum,bis_datum,Stationshoehe,geoBreite,geoLaenge,Stationsname,Bundesland
1119,19172,20200901,20221130,48,54.0246,9.388,Wacken,Schleswig-Holstein


Most robust approach probably to get the station ID, then load the list of links from the website and match. Because the end dates here do not match the end dates of the actual available data.

In [38]:
import requests
from bs4 import BeautifulSoup

def get_url_paths(url, ext='', params={}):
    response = requests.get(url, params=params)
    if response.ok:
        response_text = response.text
    else:
        return response.raise_for_status()
    soup = BeautifulSoup(response_text, 'html.parser')
    parent = [url + node.get('href') for node in soup.find_all('a') if node.get('href').endswith(ext)]
    return parent

url = DATA_DIR_URL
ext = 'zip'
url_paths = get_url_paths(url, ext)

In [39]:
url_paths[0]

'https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/monthly/kl/historicalmonatswerte_KL_00001_19310101_19860630_hist.zip'

In [49]:
stn_id = 19172
f'KL_{stn_id:05}'

'KL_19172'

In [59]:
import re
def find_url_for_station(station_id: int, url_paths: list[str]):
    exp = re.compile(f'KL_{station_id:05}')
    for url in url_paths[:20]:
        print(exp.match(url))
        
# find_url_for_station(1, url_paths)

station_id=1
exp = re.search(f'KL_{station_id:05}','blablablKL_00001_jakdjfkajfk')
print(exp)

<re.Match object; span=(8, 16), match='KL_00001'>


In [60]:
len(url_paths)

1103

Are there duplicates? Try getting the Station IDs out of the list of paths and then check for mismatch.